# Group assignment object detection

### Choice of model

In this project, the task is to determine the birds and their location inside a picture. Birds can be of many differnt shapes and forms, so it is important to have a generalizable model. Secondly, it is critical that the model can make predictions in a tractable manner. It should be fast enough to have the model running on inference on the drone's hardware. For that reason, the model can't be large and complex, it should be lightweight and apaptable. 

YOLO is a model that is known for its speed, it is fast and flexible. This will be the basis of our model's architecture. Yolo predicts a location bound for an object. 

An issue I have with the data is that I want to use apple's background removing feature but because of the file names. That are super long and complex as they are hashes, it gets confused because the hash potentially mimics a hash that is protected or another reason. So the best appraoch is to clean the file names. But I don't want to do that as it seems cumbersome as my whole code is already great as it is now. 

In [20]:
!pip install pillow opencv-python pydantic
!pip install -U "pydantic>=2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.14
    Uninstalling pydantic-1.10.14:
      Successfully uninstalled pydantic-1.10.14
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepspeed 0.10.3 requires pydantic<2.0.0, but you have pydantic 2.11.4 which is incompatible.


In [1]:
import pydantic
print(pydantic.VERSION)

2.11.4


In [5]:
from image_objects import AllImages

In [6]:
def get_related_pictures_from_file_path(label_file: str, image_paths: List[str]):
    return [file_path for file_path in image_paths if label_file in file_path]

def get_files_in_data_folder(path, all_images: AllImages):
    images_paths = [f"{path}/images/{file_path}" for file_path in os.listdir(f"{path}/images")]
    label_file_names = [file_path for file_path in os.listdir(f"{path}/labels")]
    
    for label_file in label_file_names:
        if ".DS_Store" in label_file:
            continue
        # the same picture has the first part the same but might have had different augmentation
        first_file_name = label_file.split(".")[0]
        if first_file_name in all_images.images_dict:
            image = all_images.images_dict[first_file_name]
        else:
            with open(f"{path}/labels/{label_file}") as f:
                label_text = f.read()
            image = ImageData(image_name=first_file_name,
                              label_text=label_text)
        label_file_no_ext = os.path.splitext(label_file)[0]
        found_image = [file_name for file_name in images_paths if label_file_no_ext in file_name][0]
        image.image_paths.append(found_image)
        all_images.images_dict[first_file_name] = image
    
    return all_images

def get_all_files():
    all_images = AllImages()
    test_images = get_files_in_data_folder("data/Harmful Birds Detection.v1i.yolov11/test", all_images)
    train_images = get_files_in_data_folder("data/Harmful Birds Detection.v1i.yolov11/train", all_images)
    valid_images = get_files_in_data_folder("data/Harmful Birds Detection.v1i.yolov11/valid", all_images)
    return all_images

all_images_objects = get_all_files()





### Create export of images of pigeons and crows

In [108]:
# Only needed to do this line once. After that you use apple shortcuts to process these images
#all_images_objects.copy_crows_pigeons("data/files_to_clean_nopixels")

1913


Exception: folder already exists

# Preprocess locally using Apple shortcuts

Now you made a folder with "crows" and "pigeons" as subfolders. You need to use apple shortcuts to remove the background of these pictures and add them to a folder. Do this seperately for the pigeons and crows folders. Important to create a "pigeons" and "crows" folder in this new folder. Use this new folder to add the pictures without background to the existing AllImages object

Link to Apple shortcut (Self made)
https://www.icloud.com/shortcuts/cb718c0257a8443fb68a9d5243597a47

### Recognized problem

Some of the images have instead of 4 yolo coordinates have 6 and many classifications, even though there is only a single pigeon in the bird. Therefore these are removed. These pictures are annoated with an outline around the bird, but that is not the correct format for our project

In [4]:
all_images_objects.load_removed_background_pictures("data/Subject images crows/Subjects not pixelated")

In [14]:
image = all_images_objects.get_random_cropped_images(25)



0


IndexError: Cannot choose from an empty sequence

Opencv to add the pictures to the pictures